<a href="https://colab.research.google.com/github/ArmanKhan-git/Emotion_prediction_app/blob/main/NLP_for_ML_emotion_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('train.txt',sep=';',header=None,names=['text','emotion']) #train.txt me ; seperated data hai isliye sep=; kiya wnr adefault , hota hai

In [3]:
df.head()

,text,emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [4]:
df.isnull().sum()

,0
text,0
emotion,0


# changing emotions to number


In [5]:
unique_emotion=df['emotion'].unique()

In [6]:
emotion_numbers={}
i=0
for emo in unique_emotion:
  emotion_numbers[emo]=i
  i+=1
df['emotion']=df['emotion'].map(emotion_numbers) # lekin isme labelEncoding krni chaiye thi cause easy hai

In [7]:
df.head()

,text,emotion
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,1
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,1


#  step 1) Converting data into lowercase

In [8]:
df['text']=df['text'].apply(lambda x:x.lower())

In [9]:
df.head()

,text,emotion
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,1
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,1


# step 2) removing punctuations

In [10]:
import string
def remove_punc(txt):
    return txt.translate(str.maketrans('','',string.punctuation))

In [11]:
df['text']=df['text'].apply(remove_punc)

# 3) remove numbers

In [12]:
def remove_num(txt):
  new=""
  for i in txt:
    if not i.isdigit():
      new=new+i
  return new

df['text']=df['text'].apply(remove_num)

# 4) remove urls

In [13]:
import re

def remove_urls(txt):
    return re.sub(r'http\S+|www\S+|https\S+', '', txt, flags=re.MULTILINE)
df['text']=df['text'].apply(remove_urls)

# 5) remove html tags

In [14]:
def remove_tags(txt):
    txt = re.sub(r'<.*?>', '', txt)
    return txt
df['text']=df['text'].apply(remove_tags)

# 6) remove emojis

In [15]:


def remove_emoji(txt):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags
                           u"\U00002700-\U000027BF"  # dingbats
                           u"\U0001F900-\U0001F9FF"  # supplemental symbols
                           "]+", flags=re.UNICODE)
    txt = emoji_pattern.sub(r'', txt)
    txt = re.sub(r'[^A-Za-z0-9\s]', '', txt)

    return txt
df['text']=df['text'].apply(remove_emoji)

# 7) remove stopwords

In [16]:
import nltk
from nltk.corpus import stopwords

# Download stopwords (only first time)
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def remove_stopwords(txt):
    return " ".join([word for word in txt.split() if word.lower() not in stop_words])
df['text']=df['text'].apply(remove_stopwords)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [17]:
df.head()

,text,emotion
0,didnt feel humiliated,0
1,go feeling hopeless damned hopeful around some...,0
2,im grabbing minute post feel greedy wrong,1
3,ever feeling nostalgic fireplace know still pr...,2
4,feeling grouchy,1


# Applying Bag of Words in this dataset

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer()
X_BoW=vectorizer.fit_transform(df['text'])


In [19]:
print('vocabulary',vectorizer.get_feature_names_out())
print('\n BoW Matrix:\n',X_BoW.toarray())

vocabulary ['aa' 'aaaaaaand' 'aaaaand' ... 'zum' 'zumba' 'zz']

 BoW Matrix:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [20]:
df.loc[df['text']=='aaaaaaand'] # ye sahi hai lekin kuch return ni krra ptani kyu

,text,emotion


# TF-IDF

# splitting

In [21]:
from sklearn.model_selection import train_test_split
X=df['text']
y=df['emotion']

In [22]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,stratify=y,random_state=42)

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
Bow=CountVectorizer()
TF=TfidfVectorizer()

In [24]:
X_train_Bow=Bow.fit_transform(X_train)
X_train_TF=TF.fit_transform(X_train)

In [25]:
X_test_Bow=Bow.transform(X_test) # Test me sirf transform apply krna hai
X_test_TF=TF.transform(X_test)

# APplying Naive Bayes and Logistic Regression

In [26]:
from sklearn.naive_bayes import MultinomialNB  # varient for classification tasks matlab
from sklearn.metrics import accuracy_score,classification_report


In [27]:
model_NB_BoW=MultinomialNB()
model_NB_BoW.fit(X_train_Bow,y_train)

MultinomialNB()

In [28]:
model_NB_TF=MultinomialNB()
model_NB_TF.fit(X_train_TF,y_train)

MultinomialNB()

In [29]:
y_pred_BoW=model_NB_BoW.predict(X_test_Bow)
accuracy_score(y_test,y_pred_BoW)

0.7675

In [30]:
y_pred_TF=model_NB_TF.predict(X_test_TF)
accuracy_score(y_test,y_pred_TF)

0.6621875

# Lets see how TF data performs on Logistic Regression

In [31]:
from sklearn.linear_model import LogisticRegression

In [32]:
model_LG=LogisticRegression(max_iter=1000)
model_LG.fit(X_train_TF,y_train)

LogisticRegression(max_iter=1000)

In [33]:
y_pred_log_TF=model_LG.predict(X_test_TF)
accuracy_score(y_test,y_pred_log_TF)

0.8509375

In [34]:
# 85% on logistic which is good

# lets try SVM

In [35]:
from sklearn.svm import SVC

In [36]:
model_svm=SVC()
model_svm.fit(X_train_TF,y_train)

SVC()

In [37]:
y_pred_svm_TF=model_LG.predict(X_test_TF)
accuracy_score(y_test,y_pred_svm_TF)

0.8509375

# Lets try hyperparamter Tuning

In [38]:
from sklearn.model_selection import GridSearchCV

In [39]:
classifier_LR=GridSearchCV(
    LogisticRegression(max_iter=2000),
    param_grid = {
    "C": [0.01, 0.1, 1, 10, 100],      # inverse regularization strength
    "penalty": ["l1", "l2"],           # type of regularization
    "solver": ["liblinear", "saga"],   # solvers that support l1 + l2
    "max_iter": [100, 200, 500]        # convergence iterations
},cv=5,
    return_train_score=False
)

In [40]:
classifier_SVM=GridSearchCV(
    SVC(),
    param_grid =  {
    "C": [0.1, 1, 10, 100],            # margin hardness
    "kernel": ["linear", "rbf"],       # linear works well for text, rbf if non-linear
    "gamma": ["scale", "auto"],        # only used if kernel='rbf'
},cv=5,
    return_train_score=False
)

In [41]:
classifier_LR.fit(X_train_TF,y_train)



/usr/local/lib/python3.12/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means 

GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=2000),
             param_grid={'C': [0.01, 0.1, 1, 10, 100],
                         'max_iter': [100, 200, 500], 'penalty': ['l1', 'l2'],
                         'solver': ['liblinear', 'saga']})

In [42]:
classifier_SVM.fit(X_train_TF,y_train)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100], 'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'rbf']})

In [47]:
y_pred_classifier_LR=classifier_LR.best_estimator_.predict(X_test_TF)


In [48]:
y_pred_classifier_SVM=classifier_SVM.best_estimator_.predict(X_test_TF)

In [49]:
accuracy_score(y_test,y_pred_classifier_LR)

0.89125

In [50]:
accuracy_score(y_test,y_pred_classifier_SVM)

0.8803125

# Now i m gonna get the model and vectorizer from this and use it on streamlit

In [52]:
import joblib
joblib.dump(classifier_LR.best_estimator_,'model_LR.pkl') # classifier_LR wala model 89% accuracy
joblib.dump(TF,'vectorizer_TF.pkl')  # TF-IDF wala vectorizer
joblib.dump(emotion_numbers, "emotion_mapping.pkl") # isme emotions number forms dale hai to unko thik krne ke liye ye use krenge


['emotion_mapping.pkl']